### Vamos a instalar paquetes e importar librerías para trabajar y descargar los Data Sets.

In [2]:
# Importo librerías y paquetes necesarios
import numpy
import pandas as pd
import zipfile
import os
import json
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from requests.packages.urllib3.poolmanager import PoolManager
import ssl

### Trato advertencias

In [6]:
# Deshabilita advertencias de solicitudes inseguras (SOLO para desarrollo)
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

### Llamo las rutas de los archivos ZIP y el Endpoint de la API

In [7]:
# Rutas a los archivos ZIP y el endpoint de la API
credit_card_zip_path = r"C:\Users\Esteban García\OneDrive\Escritorio\LABs\Datasets para Trabajar NoCountry\Credit Card Transactions Fraud Detection Dataset.zip"
ecommerce_zip_path = r"C:\Users\Esteban García\OneDrive\Escritorio\LABs\Datasets para Trabajar NoCountry\Fraudulent E-commerce Transactions.zip"

# Endpoint de la API del BCRA (ajustar según la documentación)
bcra_api_url = "https://api.bcra.gob.ar/estadisticas/v2.0/PrincipalesVariables"

### Establezco la ruta donde guardaré los archivos *.csv procesados

In [9]:
# Ruta donde se guardarán los archivos CSV procesados
output_dir = r"C:\Users\Esteban García\OneDrive\Escritorio\LABs\Datasets para Trabajar NoCountry"

### Defino la función

In [10]:
# Función para explorar y analizar los archivos *.zip 

def list_files_in_zip(zip_path):
    """Lista los archivos dentro de un archivo ZIP."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        return zip_ref.namelist()


def extract_and_read_zip(zip_path, file_name):
    """Extrae archivos ZIP y lee los datasets."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("temp_dataset")
        file_path = os.path.join("temp_dataset", file_name)
        return pd.read_csv(file_path)

### Verifico

In [11]:
# Verifico los nombres de los archivos dentro de los ZIPs
credit_card_files = list_files_in_zip(credit_card_zip_path)
ecommerce_files = list_files_in_zip(ecommerce_zip_path)

# Verifico la lectura correcta de los archivos
credit_card_file_name = next((file for file in credit_card_files if file.endswith('.csv')), None)
ecommerce_file_name = next((file for file in ecommerce_files if file.endswith('.csv')), None)

if credit_card_file_name:
    credit_card_df = extract_and_read_zip(credit_card_zip_path, credit_card_file_name)
else:
    print("No se encontró un archivo CSV en el ZIP de tarjetas de crédito.")

if ecommerce_file_name:
    ecommerce_df = extract_and_read_zip(ecommerce_zip_path, ecommerce_file_name)
else:
    print("No se encontró un archivo CSV en el ZIP de comercio electrónico.")


In [12]:
print(credit_card_files)
print(ecommerce_files)


['fraudTest.csv', 'fraudTrain.csv']
['Fraudulent_E-Commerce_Transaction_Data.csv', 'Fraudulent_E-Commerce_Transaction_Data_2.csv']


### Consulto la API del BCRA

In [13]:
import requests
import pandas as pd

# URL base de la API
base_url = "https://api.bcra.gob.ar/estadisticas/v2.0/"

# Endpoint para obtener principales variables
variables_url = base_url + "principalesvariables"

# Solicitar los datos de principales variables
response = requests.get(variables_url, verify=False)
if response.status_code == 200:
    variables = response.json()['results']
else:
    print("Error al obtener principales variables:", response.status_code)

# Crear un DataFrame con las principales variables
variables_df = pd.DataFrame(variables)

# Mostrar las primeras filas del DataFrame
print(variables_df.head())

# Guardar el DataFrame en un archivo CSV
output_dir = "C:/Users/Esteban García/OneDrive/Escritorio/LABs/Datasets para Trabajar NoCountry/"
variables_df.to_csv(output_dir + "principales_variables.csv", index=False)

# Función para obtener datos de una variable específica
def obtener_datos_variable(id_variable, desde, hasta):
    datos_url = f"{base_url}datosvariable/{id_variable}/{desde}/{hasta}"
    response = requests.get(datos_url, verify=False)
    if response.status_code == 200:
        return response.json()['results']
    else:
        print("Error al obtener datos de la variable:", response.status_code)
        return []

# Rango de fechas desde 2021 hasta 2024
id_variable = 1  # ID de la variable a consultar
desde = "2024-01-01"  # Fecha de inicio del rango
hasta = "2024-12-31"  # Fecha de fin del rango

datos_variable = obtener_datos_variable(id_variable, desde, hasta)

# Crear un DataFrame con los datos de la variable
datos_variable_df = pd.DataFrame(datos_variable)

# Mostrar las primeras filas del DataFrame de datos de la variable
print(datos_variable_df.head())

# Guardar el DataFrame en un archivo CSV
datos_variable_df.to_csv(output_dir + "datos_variable.csv", index=False)


   idVariable  cdSerie                                        descripcion  \
0           1      246  Reservas Internacionales del BCRA (en millones...   
1           4     7927  Tipo de Cambio Minorista ($ por USD) Comunicac...   
2           5      272  Tipo de Cambio Mayorista ($ por USD) Comunicac...   
3           6     7935             Tasa de Política Monetaria (en % n.a.)   
4           7     1222     BADLAR en pesos de bancos privados (en % n.a.)   

        fecha     valor  
0  2024-07-08  28348.00  
1  2024-07-11    956.91  
2  2024-07-11    920.50  
3  2024-07-11     40.00  
4  2024-07-10     34.25  
   idVariable       fecha    valor
0           1  2024-01-02  23470.0
1           1  2024-01-03  23677.0
2           1  2024-01-04  23835.0
3           1  2024-01-05  24119.0
4           1  2024-01-08  23233.0
